# Preparation

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline, AutoTokenizer, AutoModel
import warnings
import os
import json
import re
from datetime import datetime
from collections import defaultdict, Counter
warnings.filterwarnings('ignore')

2025-06-02 12:22:27.564898: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748866947.587391     205 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748866947.594185     205 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Load Data

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/cleaned/cleaned_all_data.csv')
df

/kaggle/input/cleaned/cleaned_all_data.csv


,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764
2,https://www.detik.com/jatim/hukum-dan-kriminal...,"Kriminalitas Surabaya Marak, Pengamat Sebut Pe...",2024-09-05 02:01:00,Sejumlah kejadian kriminalitas kian marak terj...,72,1224
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398
...,...,...,...,...,...,...
631,https://www.detik.com/sumut/hukum-dan-kriminal...,"Curi Mobil Ambulans Puskesmas, 2 Pria di Simal...",2024-09-07 19:00:00,Dua pria di Simalungun ditangkap personel kepo...,68,1778
632,https://www.detik.com/sumbagsel/hukum-dan-krim...,Polisi Sebut Pelaku Utama Pembunuh Siswi SMP d...,2024-09-07 18:00:00,Empat pelaku pembunuhan dan pemerkosaan siswi ...,75,3003
633,https://www.detik.com/sumut/hukum-dan-kriminal...,Pria di Batam Kepergok Lecehkan Tetangga-Curi ...,2024-09-07 14:29:00,Seorang pria berinisial KP (19) warga Tiban Ba...,59,2000
634,https://www.detik.com/jabar/hukum-dan-kriminal...,"Aksi 'Ninja' Curi 401 Kartu ATM Terekam CCTV, ...",2024-09-06 15:02:00,Sebanyak 401 lembar kartu ATM milik nasabah sa...,64,2347


In [5]:
df.shape

(636, 6)

# Train Model

In [3]:
pipe = pipeline("token-classification", model="cahya/NusaBert-ner-v1.3")
tokenizer = AutoTokenizer.from_pretrained("cahya/NusaBert-ner-v1.3")
model = AutoModel.from_pretrained("cahya/NusaBert-ner-v1.3")

Device set to use cuda:0


In [ ]:
def debug_nusabert_output(text, pipe):
    try:
        raw_entities = pipe(text)
        print("Raw NusaBert Output:")
        for entity in raw_entities:
            print(f"  {entity}")
        
        print("\nUnique entity labels found:")
        unique_labels = set()
        for entity in raw_entities:
            original_label = entity['entity']
            cleaned_label = original_label.replace('B-', '').replace('I-', '')
            unique_labels.add(f"{original_label} -> {cleaned_label}")
        
        for label in sorted(unique_labels):
            print(f"  {label}")
            
        return raw_entities
    except Exception as e:
        print(f"Error: {e}")
        return []

# Test dengan satu artikel
test_text = df.iloc[0]['isi_berita']
print("Testing dengan artikel pertama:")
print(f"Text: {test_text[:200]}...")
print("\n" + "="*50)

debug_output = debug_nusabert_output(test_text, pipe)

Testing dengan artikel pertama:
Text: Pelaku pembunuhan dan pemerkosaan AA (14) seorang remaja putri di Kuburan Cina Palembang sudah ditangkap dan ditetapkan tersangka. Keempat tersangka merupakan anak di bawah umur, sama seperti korbanny...

Raw NusaBert Output:
  {'entity': 'B-PER', 'score': 0.9859017, 'index': 5, 'word': 'ĠAA', 'start': 33, 'end': 36}
  {'entity': 'B-CRD', 'score': 0.99209154, 'index': 7, 'word': '14', 'start': 38, 'end': 40}
  {'entity': 'B-LOC', 'score': 0.9789508, 'index': 13, 'word': 'ĠKub', 'start': 65, 'end': 69}
  {'entity': 'I-LOC', 'score': 0.9747813, 'index': 14, 'word': 'uran', 'start': 69, 'end': 73}
  {'entity': 'I-LOC', 'score': 0.99543375, 'index': 15, 'word': 'ĠCina', 'start': 73, 'end': 78}
  {'entity': 'B-GPE', 'score': 0.59830356, 'index': 16, 'word': 'ĠPalembang', 'start': 78, 'end': 88}
  {'entity': 'B-CRD', 'score': 0.5197823, 'index': 23, 'word': 'ĠKeempat', 'start': 130, 'end': 138}
  {'entity': 'B-ORG', 'score': 0.9998621, 'index': 38, 'word

In [ ]:
CRIME_TYPES = [
    'pembunuhan', 'pemerkosaan', 'penganiayaan', 'pembacokan', 'penikaman',
    'penyiraman air keras', 'penembakan', 'pengeroyokan', 'mutilasi',
    'pencurian', 'curanmor', 'pencurian motor', 'pencurian dengan kekerasan',
    'perampokan', 'begal', 'penjambretan', 'penggelapan',
    'pencurian spesialis bongkar warung', 'bajing loncat', 'pembobolan',
    'kasus sabu', 'peredaran obat-obatan terlarang', 'kasus narkoba',
    'penyalahgunaan narkoba', 'peredaran pil koplo', 'pencabulan', 'sodomi',
    'pelecehan seksual', 'korupsi dana desa', 'korupsi', 'uang palsu',
    'tawuran', 'aksi perusakan', 'pemalsuan dokumen', 'penjualan miras ilegal',
    'perjudian', 'sabung ayam', 'penculikan', 'perdagangan orang',
    'pembakaran', 'penipuan', 'penyelundupan', 'pemalakan', 'gangster',
    'perampokan agen ATM mini', 'terorisme', 'radikalisme', 'pencucian uang',
    'pengeroyokan', 'perkelahian', 'pembegalan', 'penodongan'
]

EVIDENCE_TYPES = [
    'barang bukti', 'bukti', 'alat bukti', 'saksi mata', 'rekaman cctv',
    'rekaman video', 'foto', 'dokumen', 'surat', 'kontrak', 'kwitansi',
    'nota', 'struk', 'invoice', 'rekening bank', 'transfer', 'sms',
    'percakapan', 'chat', 'whatsapp', 'telegram', 'email', 'surat elektronik',
    'keterangan saksi', 'pengakuan', 'visum', 'autopsi', 'hasil lab',
    'tes dna', 'tes narkoba', 'sidik jari', 'jejak', 'darah', 'rambut',
    'dna', 'sertipikat', 'ijazah', 'kartu identitas', 'sim', 'stnk',
    'bpkb', 'akta', 'surat nikah', 'hard disk', 'laptop', 'handphone',
    'flashdisk', 'cd', 'dvd', 'kartu memori', 'memory card', 'cctv',
    'kamera', 'perekam', 'alat perekam', 'microphone', 'handycam'
]

def detect_crime_types(text):

    text_lower = text.lower()
    detected_crimes = []
    
    for crime in CRIME_TYPES:
        # Exact match pattern
        pattern = r'\b' + re.escape(crime.lower()) + r'\b'
        matches = re.finditer(pattern, text_lower)
        
        for match in matches:
            detected_crimes.append({
                'entity': 'CRIMETYPE',
                'word': crime,
                'start': match.start(),
                'end': match.end(),
                'score': 1.0
            })
    
    # Additional pattern-based crime detection
    crime_patterns = {
        'pembunuhan': [r'\b(membunuh|dibunuh|terbunuh|bunuh|tewas.*dibunuh)\b'],
        'pencurian': [r'\b(mencuri|dicuri|tercuri|curi|mengambil.*tanpa.*izin)\b'],
        'narkoba': [r'\b(narkoba|narkotika|psikotropika|sabu|ganja|ekstasi|kokain|heroin|shabu)\b'],
        'korupsi': [r'\b(korupsi|suap|gratifikasi|penyalahgunaan.*jabatan|mark.*up)\b'],
        'penipuan': [r'\b(menipu|ditipu|tertipu|penipuan|penipu|modus)\b'],
        'pemerkosaan': [r'\b(perkosa|memperkosa|diperkosa|cabul|mencabuli|dicabuli)\b']
    }
    
    for crime_type, patterns in crime_patterns.items():
        for pattern in patterns:
            matches = re.finditer(pattern, text_lower)
            for match in matches:
                # Check if not already detected
                is_duplicate = any(
                    abs(match.start() - existing['start']) < 5 
                    for existing in detected_crimes
                )
                if not is_duplicate:
                    detected_crimes.append({
                        'entity': 'CRIMETYPE',
                        'word': crime_type,
                        'start': match.start(),
                        'end': match.end(),
                        'score': 0.9
                    })
    
    return detected_crimes

def detect_evidence_types(text):

    text_lower = text.lower()
    detected_evidence = []
    
    for evidence in EVIDENCE_TYPES:
        pattern = r'\b' + re.escape(evidence.lower()) + r'\b'
        matches = re.finditer(pattern, text_lower)
        
        for match in matches:
            detected_evidence.append({
                'entity': 'EVIDENCE',
                'word': evidence,
                'start': match.start(),
                'end': match.end(),
                'score': 1.0
            })
    
    # Additional evidence patterns
    evidence_patterns = {
        'dokumen_resmi': [r'\b(surat.*resmi|dokumen.*resmi|akta.*resmi|sertipikat.*resmi)\b'],
        'rekaman_digital': [r'\b(rekaman.*digital|file.*digital|data.*digital)\b'],
        'keterangan_saksi': [r'\b(keterangan.*saksi|kesaksian|testimoni)\b'],
        'hasil_forensik': [r'\b(hasil.*forensik|analisis.*forensik|lab.*forensik)\b']
    }
    
    for evidence_type, patterns in evidence_patterns.items():
        for pattern in patterns:
            matches = re.finditer(pattern, text_lower)
            for match in matches:
                is_duplicate = any(
                    abs(match.start() - existing['start']) < 5 
                    for existing in detected_evidence
                )
                if not is_duplicate:
                    detected_evidence.append({
                        'entity': 'EVIDENCE',
                        'word': evidence_type.replace('_', ' '),
                        'start': match.start(),
                        'end': match.end(),
                        'score': 0.9
                    })
    
    return detected_evidence

In [ ]:
def detect_dates_pattern(text):

    import re
    detected_dates = []
    
    # Pattern untuk berbagai format tanggal Indonesia
    date_patterns = [
        r'\b\d{1,2}[\s\-/]\w+[\s\-/]\d{4}\b',  # 15 Januari 2024, 15-01-2024
        r'\b\d{1,2}[\s\-/]\d{1,2}[\s\-/]\d{4}\b',  # 15/01/2024, 15-01-2024
        r'\b\d{4}[\s\-/]\d{1,2}[\s\-/]\d{1,2}\b',  # 2024-01-15
        r'\b(?:senin|selasa|rabu|kamis|jumat|sabtu|minggu)\b',  # Hari
        r'\b(?:januari|februari|maret|april|mei|juni|juli|agustus|september|oktober|november|desember)\b',  # Bulan
        r'\b\d{1,2}\s*(?:januari|februari|maret|april|mei|juni|juli|agustus|september|oktober|november|desember)\s*\d{4}\b',  # 15 Januari 2024
        r'\bpukul\s*\d{1,2}[:.]\d{2}\b',  # pukul 14:30
        r'\b\d{1,2}[:.]\d{2}\s*(?:WIB|WITA|WIT)\b',  # 14:30 WIB
        r'\b(?:kemarin|hari\s*ini|besok|lusa)\b',  # Referensi waktu relatif
        r'\b(?:pagi|siang|sore|malam)\s*(?:hari|ini|kemarin|tadi)?\b'  # Waktu dalam hari
    ]
    
    text_lower = text.lower()
    
    for pattern in date_patterns:
        matches = re.finditer(pattern, text_lower, re.IGNORECASE)
        for match in matches:
            detected_dates.append({
                'entity': 'DAT',
                'word': text[match.start():match.end()],
                'start': match.start(),
                'end': match.end(),
                'score': 0.95
            })
    
    return detected_dates

In [ ]:
def extract_entities_nusabert_enhanced(text, pipe):
    
    try:
        entities = pipe(text)
        
        # Enhanced entity mapping
        entity_mapping = {
            'PER': 'PER',
            'PERSON': 'PER', 
            'B-PER': 'PER',
            'I-PER': 'PER',
            
            'LOC': 'LOC',
            'LOCATION': 'LOC',
            'B-LOC': 'LOC',
            'I-LOC': 'LOC',
            'GPE': 'GPE',  
            'B-GPE': 'GPE',
            'I-GPE': 'GPE',
            
            'ORG': 'NOR',  
            'ORGANIZATION': 'NOR',
            'B-ORG': 'NOR',
            'I-ORG': 'NOR',
            
            'DATE': 'DAT',
            'TIME': 'DAT', 
            'B-DATE': 'DAT',
            'I-DATE': 'DAT',
            'B-TIME': 'DAT',
            'I-TIME': 'DAT',
            
            'LAW': 'LAW',  
            'LEGAL': 'LAW',
            'B-LAW': 'LAW',
            'I-LAW': 'LAW',
            'MISC': 'LAW'  
        }
        
        filtered_entities = []
        
        for entity in entities:
            original_label = entity['entity']
            # Remove B- and I- prefixes
            clean_label = original_label.replace('B-', '').replace('I-', '')
            
            # Map to target entities
            mapped_label = entity_mapping.get(clean_label, entity_mapping.get(original_label, None))
            
            if mapped_label:
                filtered_entities.append({
                    'entity': mapped_label,
                    'word': entity['word'],
                    'start': entity['start'],
                    'end': entity['end'],
                    'score': float(entity['score'])
                })
        
        return filtered_entities
    
    except Exception as e:
        print(f"Error dalam NER: {e}")
        return []

def merge_consecutive_entities(entities):
    if not entities:
        return entities
    
    entities.sort(key=lambda x: x['start'])
    
    merged = []
    current_entity = entities[0].copy()
    
    for i in range(1, len(entities)):
        next_entity = entities[i]
        
        if (current_entity['entity'] == next_entity['entity'] and 
            current_entity['end'] >= next_entity['start'] - 3): 
            
            if current_entity['word'].endswith('##') or next_entity['word'].startswith('##'):
                current_word = current_entity['word'].replace('##', '')
                next_word = next_entity['word'].replace('##', '')
                current_entity['word'] = current_word + next_word
            else:
                current_entity['word'] += ' ' + next_entity['word']
            
            current_entity['end'] = next_entity['end']
            current_entity['score'] = float((current_entity['score'] + next_entity['score']) / 2)
        else:
            merged.append(current_entity)
            current_entity = next_entity.copy()
    
    merged.append(current_entity)
    return merged

In [ ]:
def extract_all_entities_enhanced(text, pipe):
    nusabert_entities = extract_entities_nusabert_enhanced(text, pipe)
    
    crime_entities = detect_crime_types(text)
    evidence_entities = detect_evidence_types(text)
    date_entities = detect_dates_pattern(text)  # Tambah deteksi DATE
    
    all_entities = nusabert_entities + crime_entities + evidence_entities + date_entities
    
    all_entities.sort(key=lambda x: x['start'])
    
    filtered_entities = []
    for entity in all_entities:
        is_overlap = False
        for existing in filtered_entities:
            if (entity['start'] < existing['end'] and entity['end'] > existing['start']):
                if entity['score'] <= existing['score']:
                    is_overlap = True
                    break
        
        if not is_overlap:
            filtered_entities = [e for e in filtered_entities 
                               if not (entity['start'] < e['end'] and entity['end'] > e['start'] and entity['score'] > e['score'])]
            filtered_entities.append(entity)
    
    entity_groups = defaultdict(list)
    for entity in filtered_entities:
        entity_groups[entity['entity']].append(entity)
    
    merged_entities = []
    for entity_type, entities in entity_groups.items():
        if entity_type in ['CRIMETYPE', 'EVIDENCE', 'DAT']:  
            merged_entities.extend(entities)
        else:
            merged = merge_consecutive_entities(entities)
            merged_entities.extend(merged)
    
    # Final sort by position
    merged_entities.sort(key=lambda x: x['start'])
    
    return merged_entities

print("Testing DATE detection standalone:")
test_text_with_dates = "Kejadian ini terjadi pada tanggal 15 Januari 2024 pukul 14:30 WIB di Jakarta kemarin sore."
date_entities = detect_dates_pattern(test_text_with_dates)
print(f"Found {len(date_entities)} dates:")
for entity in date_entities:
    print(f"  - {entity['word']} (score: {entity['score']})")

print("\nTesting full extraction:")
all_entities = extract_all_entities_enhanced(test_text_with_dates, pipe)
dat_entities = [e for e in all_entities if e['entity'] == 'DAT']
print(f"DAT entities in full extraction: {len(dat_entities)}")
for entity in dat_entities:
    print(f"  - {entity['word']} (score: {entity['score']})")

Testing DATE detection standalone:
Found 7 dates:
  - 15 Januari 2024 (score: 0.95)
  - Januari (score: 0.95)
  - 15 Januari 2024 (score: 0.95)
  - pukul 14:30 (score: 0.95)
  - 14:30 WIB (score: 0.95)
  - kemarin (score: 0.95)
  - sore (score: 0.95)

Testing full extraction:
DAT entities in full extraction: 4
  - 15 Januari 2024 (score: 0.95)
  - pukul 14:30 (score: 0.95)
  - kemarin (score: 0.95)
  - sore (score: 0.95)


In [ ]:
def process_dataframe_entities_enhanced(df, pipe, sample_size=None):
    if sample_size:
        df_sample = df.head(sample_size)
    else:
        df_sample = df
    
    results = []
    target_entities = ['GPE', 'LOC', 'NOR', 'LAW', 'DAT', 'PER', 'CRIMETYPE', 'EVIDENCE']
    
    for idx, row in df_sample.iterrows():
        if idx % 100 == 0:
            print(f"Processing artikel {idx+1}/{len(df_sample)}")
        
        try:
            entities = extract_all_entities_enhanced(row['isi_berita'], pipe)
            
            # Group entities by type
            entity_dict = defaultdict(list)
            for entity in entities:
                if entity['entity'] in target_entities:
                    entity_dict[entity['entity']].append({
                        'word': entity['word'],
                        'start': int(entity['start']),
                        'end': int(entity['end']),
                        'score': float(entity['score'])
                    })
            
            result = {
                'index': int(idx),
                'url': row['url'],
                'judul': row['judul'],
                'tanggal': row['tanggal'],
                'entities': dict(entity_dict),
                'total_entities': len(entities)
            }
            
            results.append(result)
            
        except Exception as e:
            print(f"Error processing article {idx}: {e}")
            continue
    
    return results

def save_enhanced_results(results, base_filename='enhanced_entity_results'):
    
    flattened_data = []
    target_entities = ['GPE', 'LOC', 'NOR', 'LAW', 'DAT', 'PER', 'CRIMETYPE', 'EVIDENCE']
    
    for result in results:
        base_row = {
            'index': result['index'],
            'url': result['url'],
            'judul': result['judul'],
            'tanggal': result['tanggal'],
            'total_entities': result['total_entities']
        }
        
        for entity_type in target_entities:
            entities = result['entities'].get(entity_type, [])
            base_row[f'{entity_type.lower()}_count'] = len(entities)
            base_row[f'{entity_type.lower()}_entities'] = '; '.join([e['word'] for e in entities])
            
            if entities:
                scores = [e['score'] for e in entities]
                base_row[f'{entity_type.lower()}_avg_score'] = round(sum(scores) / len(scores), 3)
                base_row[f'{entity_type.lower()}_max_score'] = round(max(scores), 3)
            else:
                base_row[f'{entity_type.lower()}_avg_score'] = 0
                base_row[f'{entity_type.lower()}_max_score'] = 0
        
        flattened_data.append(base_row)
    
    # Save CSV
    df_results = pd.DataFrame(flattened_data)
    df_results.to_csv(f'{base_filename}.csv', index=False, encoding='utf-8')
    
    print(f"Enhanced results saved to {base_filename}.csv")
    print(f"Total articles processed: {len(results)}")
    
    return df_results

In [12]:
print("Testing enhanced entity extraction dengan sample...")
test_text = df.iloc[0]['isi_berita']
print(f"Test text preview: {test_text[:300]}...")
print("\n" + "="*50)

test_entities = extract_all_entities_enhanced(test_text, pipe)
print(f"\nTotal entities found: {len(test_entities)}")

# Group by entity type
entity_summary = defaultdict(list)
for entity in test_entities:
    entity_summary[entity['entity']].append(entity['word'])

print("\nEntity summary:")
for entity_type, words in entity_summary.items():
    print(f"  {entity_type}: {len(words)} entities")
    print(f"    Examples: {', '.join(words[:5])}")

Testing enhanced entity extraction dengan sample...
Test text preview: Pelaku pembunuhan dan pemerkosaan AA (14) seorang remaja putri di Kuburan Cina Palembang sudah ditangkap dan ditetapkan tersangka. Keempat tersangka merupakan anak di bawah umur, sama seperti korbannya. Pengamat Hukum dari Universitas Taman Siswa Palembang, Dr Azwar Agus menyampaikan turut prihatin ...


Total entities found: 12

Entity summary:
  CRIMETYPE: 3 entities
    Examples: pembunuhan, pemerkosaan, pembunuhan
  PER: 3 entities
    Examples: ĠAA, ĠDr ĠAzwar ĠAgus, ĠAzwar
  LOC: 1 entities
    Examples: ĠKub uran ĠCina
  GPE: 1 entities
    Examples: ĠPalembang
  NOR: 1 entities
    Examples: ĠUniversitas ĠTaman ĠSiswa ĠPalembang
  DAT: 2 entities
    Examples: Kamis, 5/9/2024
  EVIDENCE: 1 entities
    Examples: handphone


In [13]:
print("Processing sample data (first 100 articles)...")
sample_results = process_dataframe_entities_enhanced(df, pipe, sample_size=100)
sample_output = save_enhanced_results(sample_results, 'sample_enhanced_results')

print("\nSample results preview:")
print(sample_output.head())

Processing sample data (first 100 articles)...
Processing artikel 1/100


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Enhanced results saved to sample_enhanced_results.csv
Total articles processed: 100

Sample results preview:
   index                                                url  \
0      0  https://www.detik.com/sumbagsel/hukum-dan-krim...   
1      1  https://www.detik.com/jatim/hukum-dan-kriminal...   
2      2  https://www.detik.com/jatim/hukum-dan-kriminal...   
3      3  https://www.detik.com/sumut/hukum-dan-kriminal...   
4      4  https://www.detik.com/sumut/hukum-dan-kriminal...   

                                               judul              tanggal  \
0  4 Anak di Bawah Umur Tersangka Pembunuhan di P...  2024-09-05 20:30:00   
1  26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...  2024-09-05 18:38:00   
2  Kriminalitas Surabaya Marak, Pengamat Sebut Pe...  2024-09-05 02:01:00   
3  Pekerja Kafe Disiram Air Keras hingga Wajah 'B...  2024-09-04 21:40:00   
4  Hilang Nyawa Pria di Simalungun gegara Rebutan...  2024-09-03 09:03:00   

   total_entities  gpe_count  \
0              12

In [14]:
print("Processing all articles...")
all_results = process_dataframe_entities_enhanced(df, pipe)
enhanced_output = save_enhanced_results(all_results, 'all_enhanced_entity_results')

Processing all articles...
Processing artikel 1/636
Processing artikel 101/636
Processing artikel 201/636
Processing artikel 301/636
Processing artikel 401/636
Processing artikel 501/636
Processing artikel 601/636
Enhanced results saved to all_enhanced_entity_results.csv
Total articles processed: 636


In [ ]:
def analyze_enhanced_entity_statistics(output_df):

    print("=== ENHANCED ENTITY STATISTICS ===\n")
    
    target_entities = ['gpe', 'loc', 'nor', 'law', 'dat', 'per', 'crimetype', 'evidence']
    total_articles = len(output_df)
    
    summary_data = []
    
    for entity_type in target_entities:
        count_col = f'{entity_type}_count'
        entities_col = f'{entity_type}_entities'
        score_col = f'{entity_type}_avg_score'
        
        if count_col in output_df.columns:
            total_entities = output_df[count_col].sum()
            articles_with_entity = len(output_df[output_df[count_col] > 0])
            coverage = (articles_with_entity / total_articles) * 100
            avg_per_article = total_entities / total_articles
            avg_score = output_df[score_col].mean() if score_col in output_df.columns else 0
            
            summary_data.append({
                'Entity_Type': entity_type.upper(),
                'Total_Entities': int(total_entities),
                'Articles_with_Entity': articles_with_entity,
                'Coverage_Percentage': round(coverage, 1),
                'Avg_per_Article': round(avg_per_article, 2),
                'Avg_Confidence_Score': round(avg_score, 3),
                'Max_in_Single_Article': int(output_df[count_col].max())
            })
            
            print(f"{entity_type.upper()}:")
            print(f"  Total entities: {total_entities}")
            print(f"  Coverage: {coverage:.1f}% ({articles_with_entity}/{total_articles} articles)")
            print(f"  Average per article: {avg_per_article:.2f}")
            print(f"  Average confidence: {avg_score:.3f}")
            print()
    
    # Create summary DataFrame
    summary_df = pd.DataFrame(summary_data)
    print("=== SUMMARY TABLE ===")
    print(summary_df.to_string(index=False))
    
    return summary_df

# Analyze enhanced results
if 'enhanced_output' in locals():
    enhanced_stats = analyze_enhanced_entity_statistics(enhanced_output)
else:
    enhanced_stats = analyze_enhanced_entity_statistics(sample_output)

=== ENHANCED ENTITY STATISTICS ===

GPE:
  Total entities: 2396
  Coverage: 96.9% (616/636 articles)
  Average per article: 3.77
  Average confidence: 0.912

LOC:
  Total entities: 1662
  Coverage: 84.9% (540/636 articles)
  Average per article: 2.61
  Average confidence: 0.703

NOR:
  Total entities: 861
  Coverage: 55.5% (353/636 articles)
  Average per article: 1.35
  Average confidence: 0.440

LAW:
  Total entities: 452
  Coverage: 31.6% (201/636 articles)
  Average per article: 0.71
  Average confidence: 0.266

DAT:
  Total entities: 3495
  Coverage: 99.5% (633/636 articles)
  Average per article: 5.50
  Average confidence: 0.946

PER:
  Total entities: 7270
  Coverage: 99.7% (634/636 articles)
  Average per article: 11.43
  Average confidence: 0.918

CRIMETYPE:
  Total entities: 2556
  Coverage: 85.2% (542/636 articles)
  Average per article: 4.02
  Average confidence: 0.827

EVIDENCE:
  Total entities: 927
  Coverage: 58.6% (373/636 articles)
  Average per article: 1.46
  Averag

In [ ]:
def analyze_specific_enhanced_entities(output_df):

    print("=== DETAILED ENHANCED ENTITY ANALYSIS ===\n")
    
    target_entities = ['gpe', 'loc', 'nor', 'law', 'dat', 'per', 'crimetype', 'evidence']
    
    for entity_type in target_entities:
        entities_col = f'{entity_type}_entities'
        
        if entities_col in output_df.columns:
            print(f"{entity_type.upper()} Analysis:")
            
            # Collect all entities
            all_entities = []
            for entities_str in output_df[entities_col].dropna():
                if entities_str and entities_str.strip() != '':
                    entities = [e.strip() for e in entities_str.split(';') if e.strip()]
                    all_entities.extend(entities)
            
            if all_entities:
                entity_freq = Counter(all_entities)
                
                print(f"  Total unique {entity_type}s: {len(entity_freq)}")
                print(f"  Total {entity_type} mentions: {len(all_entities)}")
                print(f"  Most common {entity_type}s (top 10):")
                
                for entity, freq in entity_freq.most_common(10):
                    print(f"    {entity}: {freq} times")
                print()
            else:
                print(f"  No {entity_type}s detected")
                print()

# Analyze specific entities
if 'enhanced_output' in locals():
    analyze_specific_enhanced_entities(enhanced_output)
else:
    analyze_specific_enhanced_entities(sample_output)

=== DETAILED ENHANCED ENTITY ANALYSIS ===

GPE Analysis:
  Total unique gpes: 1320
  Total gpe mentions: 2396
  Most common gpes (top 10):
    ĠSurabaya: 55 times
    ĠBali: 45 times
    ĠKota ĠMedan: 41 times
    ĠSumut: 32 times
    ĠDesa: 25 times
    ĠKota ĠMedan ĠSumatera ĠUtara Sumut: 24 times
    ĠRusia: 20 times
    ĠMalaysia: 20 times
    ĠGarut: 17 times
    ĠAceh: 17 times

LOC Analysis:
  Total unique locs: 1150
  Total loc mentions: 1662
  Most common locs (top 10):
    Ġpolisi: 36 times
    ĠPolsek: 15 times
    ĠJalan ĠAH ĠNasution: 11 times
    Ġsawit: 8 times
    Ġdua: 7 times
    ĠPolresta: 7 times
    ĠJalan ĠB yp ass ĠNgurah ĠRai: 7 times
    ĠJalan ĠJamin ĠGinting: 7 times
    ĠKodam ĠI: 7 times
    ĠBarisan: 7 times

NOR Analysis:
  Total unique nors: 391
  Total nor mentions: 861
  Most common nors (top 10):
    Ġdetik Sumut: 98 times
    Ġdetik Bali: 42 times
    Ġdetik S umb ag sel: 34 times
    Ġdetik Jatim: 32 times
    Ġdetik Jabar: 30 times
    Ġdetik News:

In [ ]:
def create_final_entity_report(output_df):

    print("=== FINAL ENHANCED ENTITY EXTRACTION REPORT ===\n")
    
    target_entities = ['gpe', 'loc', 'nor', 'law', 'dat', 'per', 'crimetype', 'evidence']
    total_articles = len(output_df)
    
    # Overall statistics
    total_entities_detected = sum([output_df[f'{entity}_count'].sum() for entity in target_entities if f'{entity}_count' in output_df.columns])
    articles_with_any_entity = len(output_df[output_df[[f'{entity}_count' for entity in target_entities if f'{entity}_count' in output_df.columns]].sum(axis=1) > 0])
    
    print(f"OVERALL STATISTICS:")
    print(f"  Total articles processed: {total_articles}")
    print(f"  Total entities detected: {total_entities_detected}")
    print(f"  Articles with entities: {articles_with_any_entity} ({articles_with_any_entity/total_articles*100:.1f}%)")
    print(f"  Average entities per article: {total_entities_detected/total_articles:.2f}")
    print()
    
    # Detailed breakdown
    entity_descriptions = {
        'gpe': 'Geopolitical Entities (countries, cities, states)',
        'loc': 'Locations (places, addresses, geographical locations)',
        'nor': 'Organizations (companies, institutions, groups)',
        'law': 'Legal entities (laws, regulations, legal terms)',
        'dat': 'Dates and times (temporal expressions)',
        'per': 'Persons (names of people)',
        'crimetype': 'Crime types (custom detected criminal activities)',
        'evidence': 'Evidence types (custom detected evidence items)'
    }
    
    print("ENTITY TYPE BREAKDOWN:")
    for entity_type in target_entities:
        count_col = f'{entity_type}_count'
        if count_col in output_df.columns:
            total = output_df[count_col].sum()
            coverage = len(output_df[output_df[count_col] > 0])
            percentage = (coverage / total_articles) * 100
            
            print(f"\n{entity_type.upper()} - {entity_descriptions[entity_type]}")
            print(f"  Total detected: {total}")
            print(f"  Article coverage: {coverage}/{total_articles} ({percentage:.1f}%)")
            print(f"  Average per article: {total/total_articles:.2f}")
    
    # Success metrics
    highly_detected_entities = [entity for entity in target_entities 
                              if f'{entity}_count' in output_df.columns and 
                              len(output_df[output_df[f'{entity}_count'] > 0]) / total_articles > 0.5]
    
    print(f"\nHIGH DETECTION RATE ENTITIES (>50% coverage):")
    for entity in highly_detected_entities:
        coverage = len(output_df[output_df[f'{entity}_count'] > 0]) / total_articles * 100
        print(f"  {entity.upper()}: {coverage:.1f}% coverage")
    
    return {
        'total_articles': total_articles,
        'total_entities': total_entities_detected,
        'articles_with_entities': articles_with_any_entity,
        'high_coverage_entities': highly_detected_entities
    }

# Generate final report
if 'enhanced_output' in locals():
    final_report = create_final_entity_report(enhanced_output)
    print("\n" + "="*60)
    print("Enhanced entity extraction completed successfully!")
    print("Files saved: all_enhanced_entity_results.json and all_enhanced_entity_results.csv")
else:
    final_report = create_final_entity_report(sample_output)
    print("\n" + "="*60)
    print("Sample enhanced entity extraction completed!")
    print("Files saved: sample_enhanced_results.json and sample_enhanced_results.csv")

=== FINAL ENHANCED ENTITY EXTRACTION REPORT ===

OVERALL STATISTICS:
  Total articles processed: 636
  Total entities detected: 19619
  Articles with entities: 636 (100.0%)
  Average entities per article: 30.85

ENTITY TYPE BREAKDOWN:

GPE - Geopolitical Entities (countries, cities, states)
  Total detected: 2396
  Article coverage: 616/636 (96.9%)
  Average per article: 3.77

LOC - Locations (places, addresses, geographical locations)
  Total detected: 1662
  Article coverage: 540/636 (84.9%)
  Average per article: 2.61

NOR - Organizations (companies, institutions, groups)
  Total detected: 861
  Article coverage: 353/636 (55.5%)
  Average per article: 1.35

LAW - Legal entities (laws, regulations, legal terms)
  Total detected: 452
  Article coverage: 201/636 (31.6%)
  Average per article: 0.71

DAT - Dates and times (temporal expressions)
  Total detected: 3495
  Article coverage: 633/636 (99.5%)
  Average per article: 5.50

PER - Persons (names of people)
  Total detected: 7270
  

In [ ]:
def detect_dates_pattern(text):

    import re
    detected_dates = []
    
    # Pattern untuk berbagai format tanggal Indonesia
    date_patterns = [
        r'\b\d{1,2}[\s\-/]\w+[\s\-/]\d{4}\b',  # 15 Januari 2024, 15-01-2024
        r'\b\d{1,2}[\s\-/]\d{1,2}[\s\-/]\d{4}\b',  # 15/01/2024, 15-01-2024
        r'\b\d{4}[\s\-/]\d{1,2}[\s\-/]\d{1,2}\b',  # 2024-01-15
        r'\b(?:senin|selasa|rabu|kamis|jumat|sabtu|minggu)\b',  # Hari
        r'\b(?:januari|februari|maret|april|mei|juni|juli|agustus|september|oktober|november|desember)\b',  # Bulan
        r'\b\d{1,2}\s*(?:januari|februari|maret|april|mei|juni|juli|agustus|september|oktober|november|desember)\s*\d{4}\b',  # 15 Januari 2024
        r'\bpukul\s*\d{1,2}[:.]\d{2}\b',  # pukul 14:30
        r'\b\d{1,2}[:.]\d{2}\s*(?:WIB|WITA|WIT)\b',  # 14:30 WIB
        r'\b(?:kemarin|hari\s*ini|besok|lusa)\b',  # Referensi waktu relatif
        r'\b(?:pagi|siang|sore|malam)\s*(?:hari|ini|kemarin|tadi)?\b'  # Waktu dalam hari
    ]
    
    text_lower = text.lower()
    
    for pattern in date_patterns:
        matches = re.finditer(pattern, text_lower, re.IGNORECASE)
        for match in matches:
            detected_dates.append({
                'entity': 'DAT',
                'word': text[match.start():match.end()],
                'start': match.start(),
                'end': match.end(),
                'score': 0.95
            })
    
    return detected_dates

# Model Evaluation